In [50]:
import os,json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [5]:
load_dotenv(override = True)
api_key = os.getenv('OPENAI_API_KEY')
openai=OpenAI()

In [10]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination):
    city = destination.lower()
    price = ticket_prices.get(city)
    return f"Ticket Price for {city}: {price}"
    

In [11]:
get_ticket_price('paris')

'Ticket Price for paris: $899'

In [12]:
system_message = "You are a helpful assistant used for airline services for AIAirline"
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [19]:
price_function = {
    "name" : "get_ticket_price",
    "description" : "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "destination" : {
                "type" : "string",
                "description" : "The city that the customer wants to travel to",
            },
        },
        "required" : ["destination"],
        "additionalProperties": False
    }
}
    

In [20]:
tools = [ {"type" : "function", "function":price_function}]

In [53]:
def handle_tool_call(message):
    tool_calls = message.tool_calls[0]
    arguments = json.loads(tool_calls.function.arguments)
    city = arguments.get('destination')
    price = get_ticket_price(city)
    response = {
        'role': 'tool',
        'content' : json.dumps({'destination':city,'price':price}),
        'tool_call_id' : tool_calls.id
    }
    return response
    

In [54]:
def chat(message,history):
    messages = [
        {"role":"system","content":system_message}     
    ]
    messages += history 
    messages += [{"role":"user","content":message}]
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        tools = tools
    )
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response  = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(
            model = 'gpt-4o-mini',
            messages = messages
        )
    return response.choices[0].message.content

    

In [55]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
